In [1]:
import os
os.chdir('../..')
from pipelines.util import *
import pandas as pd
from datetime import datetime

In [2]:
data = pd.read_csv(os.path.join(WDIR, 'true-north/true_north_may_2024_clean.csv'))

In [3]:
data.describe()

,last_updated,Create Date,Do you feel the True North report identified the key challenges and opportunities facing the region?,Are you interested in attending future True North events?,Are you currently a B Corp or in the process of becoming a B Corp?,Would you be interested in hearing more from Brabners about the B Corp process?,Company name,City,Industry,sector,location,Which theme of the True North report do you most identify with and could support activity around?,How would you like to be involved with the True North network?,company_size
count,188,188,188,187,186,186,176,141,132,188,47,188,47,47
unique,177,161,2,2,2,2,165,70,50,146,19,13,7,7
top,2022-03-28 00:29:00,2022-03-28 00:29:00,Yes,Yes,No,No,Brabners LLP,London,Legal Services,Legal,Lancashire,"People, skills and the future",Keeping informed about the latest True North n...,0-9
freq,7,21,181,185,157,100,5,19,12,7,9,58,18,21


In [4]:
company_names = pd.Series(data['Company name'].sort_values())
print(company_names.unique())

['2030hub' 'AHR' 'Abeceder' 'Access Creative College'
 'Accountable Recruitment' 'Advanced Manufacturing Research Centre'
 'Agent Marketing Ltd' 'Agile Automations' 'Armstrong Watson LLP' 'Azets'
 'B-Engineering Group' 'BHP LLP'
 'Bank of England (Agency for the North West)' 'Barton Legal Limited'
 'Bicical' 'Big Hand 4 Business' 'Blackpool Council' 'Brabners LLP'
 'Bruntwood' 'Business Doctors' 'Business Health Institute'
 'Business of Science' 'CBI' 'Carbon Happy World' 'Cazenove Capital'
 'Chris Shakespeare Coaching' 'Codurance' 'Cortus Advisory Group Limited'
 'Coverdale Barclay' 'DES Group UK' 'Daneel Williams' 'Deloitte LLP'
 'E-Rail' 'ESGmark' 'Elevategm' 'Endless LLP' 'Essential-Sourcing' 'F-LEX'
 'Farrans (Construction) Limited' 'Finance Yorkshire' 'Fleetsolve Ltd'
 'Fort Search' 'Fox Brothers' 'Fragomen' 'Freshfield'
 'Giles Metcalfe Digital' 'Go-International' 'Gripple Limited'
 'Group Rhodes' 'Groupsignature' 'HSBC' 'Handelsbanken'
 'Harrowells Limited' 'Hawthorne & Burman'

There is LLoyds banking group and lloyds bank plc - we're assuming these are different companies.

In [5]:
# convert the month column to a datetime object
data['month'] = pd.to_datetime(data['Create Date'])

#convert the item to a formatted value in yyyy-mm format.
data['month_formatted'] = data['month'].apply(datetime.strftime, format='%Y-%m').sort_index(ascending=True)

In [6]:
# take the number of members and calculate the number that joined each month.
monthly_members = pd.DataFrame(data['month_formatted'].value_counts(ascending=False)).reset_index()

In [7]:
monthly_orgs = data.drop_duplicates(subset='Company name', keep='last')
monthly_orgs = pd.DataFrame(monthly_orgs['month_formatted'].value_counts(ascending=False)).reset_index()

In [8]:
def decimal_date(data):
    # make a unix timestamp column
    data['timestamp'] = pd.to_datetime(data['month_formatted'], format='%Y-%m').astype(int) / 10**9
    # make a decimal date and round to 2dp.
    data['year'] = data['timestamp'].div((86400*365.25)).add(1970).round(2)
    # drop the timestamp column
    data.drop(columns='timestamp', inplace=True) 
    # set year and formatted month as the index so they aren't included in the cumsum.
    data.set_index(['year', 'month_formatted'], inplace=True, append=True)

    return data

In [9]:
def calculate_cumsum(data, count_name):
    # order by date, then do the cumsum. reset the index, drop the original index column as not needed
    data = pd.DataFrame(data.sort_index(level=2).cumsum().reset_index().drop(columns='level_0'))
    data.rename(columns={'count': f'{count_name}'}, inplace=True)
    return data

In [10]:
# apply above functions to data
cs_monthly_members = calculate_cumsum(decimal_date(monthly_members), count_name='individuals')
cs_monthly_orgs = calculate_cumsum(decimal_date(monthly_orgs), count_name='orgs')

In [11]:
cs_merged = cs_monthly_members.merge(cs_monthly_orgs, how='inner', on=['year', 'month_formatted'])

In [12]:
# write to file
# cs_monthly_members.to_csv(os.path.join(SRC_DIR,'themes/true-north/membership/_data/cumsum_members.csv'), index=False)
# cs_monthly_orgs.to_csv(os.path.join(SRC_DIR,'themes/true-north/membership/_data/cumsum_organisations.csv'), index=False)
cs_merged.to_csv(os.path.join(SRC_DIR,'themes/true-north/membership/_data/cumsum.csv'), index=False)